# Librerías

In [1]:
# import libraries
import pandas as pd
import os
from io import StringIO
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import numpy as np
import wikipediaapi


# Cargar y unir datasets

## Cargar todos los csv


In [2]:
DATA_DIR = "raw_data"

FILES = {
    "municipios_base": "municipios_madrid_menores_50000.csv",
    "centros_salud": "recursos-y-asistencia-sanitaria.-centros-por-tipo-de-centro.-municipios.csv",
    "farmacias": "recursos-sanitarios.-farmacias.csv",
    "google_places": "google_places_municipios.csv",
    "paro_total": "total-paro-registrado.csv",
    "paro_por100": "paro-registrado-por-100-habitantes.csv",
    "cultura": "numero de intereses culturales.csv",
    "distancia": "distancia-a-la-capital_2016.csv",
    "vivienda": "IPVA 2023 VIVIENDA.csv",
    "afiliados": "afiliados-por-lugar-de-residencia-y-actividad.-municipios.csv",
    "alumnos": "alumnos-no-universitarios-del-regimen-general-matriculados-en-centros-escolares-por-tipo-de-cent.csv"
}

# === FUNCIONES ===
def cargar_csv_detectando_sep(path):
    """Carga CSV detectando separador y codificación automáticamente."""
    for sep in [",", ";"]:
        try:
            df_test = pd.read_csv(path, sep=sep, nrows=5)
            if len(df_test.columns) > 1:
                full_df = pd.read_csv(path, sep=sep)
                return full_df, sep, None
        except Exception:
            continue
    for encoding in ["utf-8", "ISO-8859-1"]:
        for sep in [",", ";"]:
            try:
                full_df = pd.read_csv(path, sep=sep, encoding=encoding)
                if len(full_df.columns) > 1:
                    return full_df, sep, encoding
            except Exception:
                continue
    return None, None, None

def explorar_dataset(nombre, path):
    """Carga y muestra información básica de un dataset."""
    print(f"\n {nombre} — {path}")
    df, sep, encoding = cargar_csv_detectando_sep(path)
    
    if df is None:
        print(" No se pudo cargar el archivo correctamente.")
        return None
    
    print(f" Separador: '{sep}' | Codificación: '{encoding}' | Filas: {len(df)} | Columnas: {len(df.columns)}")
    print(" Columnas:", list(df.columns)[:8], "..." if len(df.columns) > 8 else "")
    print(df.head(3))
    return df

def inspeccionar_texto(path, n=8):
    """Muestra algunas líneas del archivo en bruto para inspección manual."""
    print(f"\n Inspeccionando texto bruto de {path}:")
    try:
        with open(path, "r", encoding="utf-8") as f:
            for i, line in enumerate(f):
                print(line.strip())
                if i >= n:
                    break
    except UnicodeDecodeError:
        print(" Error de codificación UTF-8, probando ISO-8859-1...")
        with open(path, "r", encoding="ISO-8859-1") as f:
            for i, line in enumerate(f):
                print(line.strip())
                if i >= n:
                    break

# --- CARGA PRINCIPAL ---
def main():
    dfs = {}
    for name, filename in FILES.items():
        path = os.path.join(DATA_DIR, filename)
        if not os.path.exists(path):
            print(f" Archivo no encontrado: {path}")
            continue
        df = explorar_dataset(name, path)
        if df is not None:
            dfs[name] = df
        else:
            inspeccionar_texto(path)
    print("\n Exploración completa. DataFrames cargados:", list(dfs.keys()))
    return dfs

if __name__ == "__main__":
    dfs = main()



 municipios_base — raw_data\municipios_madrid_menores_50000.csv
 Separador: ',' | Codificación: 'None' | Filas: 155 | Columnas: 6
 Columnas: ['cod_municipio', 'municipio', 'latitud', 'longitud', 'altitud', 'poblacion'] 
   cod_municipio          municipio   latitud  longitud    altitud  poblacion
0             14       Acebeda (La)  41.08697 -3.624634  1266.5420       68.0
1             29            Ajalvir  40.53437 -3.481002   680.1722     4946.0
2             35  Alameda del Valle  40.91790 -3.843788  1109.9340      256.0

 centros_salud — raw_data\recursos-y-asistencia-sanitaria.-centros-por-tipo-de-centro.-municipios.csv
 Separador: ',' | Codificación: 'None' | Filas: 573 | Columnas: 7
 Columnas: ['Año', 'Tipo territorio', 'Código territorio', 'Territorio', 'Tipo', 'Medida', 'Valor'] 
    Año      Tipo territorio  Código territorio           Territorio  \
0  2024  Comunidad de Madrid                NaN  Comunidad de Madrid   
1  2024           Municipios               14.0      

El archivo "numero de intereses culturales.csv" no sigue el formato de un csv regular, presenta 22 columnas pero hay un punto y coma extra en el final de cada fila. Adem'as, las conas son para numeros decimales y la primera y segunda fila son descriptivas. Por lo que se eliminaría las primeras filas, limpiar texto quitando el ; final y luego las cuatro primeras columnas los nombraríamos: "Tipo territorio", "Código territorio", "Territorio". A esto le añadimos las columnas años que van desde 2006 hasta el 2024. 

In [3]:
def cargar_cultura_limpia(path):
    """Carga y limpia el dataset de cultura (corrige ';' extra y pasa de ancho a largo)."""
    with open(path, "rb") as f:
        raw = f.read()

    # eliminar BOM si existe
    if raw.startswith(b'\xef\xbb\xbf'):
        raw = raw[len(b'\xef\xbb\xbf'):]
    text = raw.decode("ISO-8859-1", errors="replace")

    # limpiar saltos de línea
    lines = [ln.rstrip("\r\n") for ln in text.splitlines()]
    # saltar dos primeras líneas descriptivas
    data_lines = lines[2:]

    cleaned = []
    for ln in data_lines:
        ln = ln.replace('\ufeff', '').replace('\u200b', '').strip()
        # eliminar cualquier ';' extra al final
        while ln.endswith(';'):
            ln = ln[:-1]
        cleaned.append(ln)

    # detectar columnas esperadas (primer fila válida)
    expected_cols = None
    for ln in cleaned:
        parts = ln.split(';')
        if len(parts) >= 3:
            expected_cols = len(parts)
            break

    if expected_cols is None:
        raise ValueError("No se pudo inferir el número de columnas del fichero de cultura.")

    normalized = []
    for ln in cleaned:
        parts = ln.split(';')
        # eliminar columnas vacías extra al final
        while len(parts) > expected_cols and parts[-1] == '':
            parts = parts[:-1]
        # recortar o rellenar según corresponda
        if len(parts) > expected_cols:
            parts = parts[:expected_cols-1] + [';'.join(parts[expected_cols-1:])]
        if len(parts) < expected_cols:
            parts += [''] * (expected_cols - len(parts))
        normalized.append(';'.join(parts))

    # encabezado correcto
    years = [str(y) for y in range(2006, 2025)]
    header = ["Tipo territorio", "Código territorio", "Territorio"] + years
    csv_text = ";".join(header) + "\n" + "\n".join(normalized)

    # leer con pandas
    df = pd.read_csv(StringIO(csv_text), sep=';', decimal=',', dtype=str, engine='python')

    # quedarnos con los municipios
    df = df[df["Tipo territorio"] == "Municipios"]

    # pasar a formato largo
    df_long = df.melt(
        id_vars=["Código territorio", "Territorio"],
        var_name="Año",
        value_name="Valor"
    )

    # limpiar tipos
    df_long["Año"] = pd.to_numeric(df_long["Año"], errors="coerce").astype("Int64")
    df_long["Valor"] = (
        df_long["Valor"]
        .str.replace(".", "", regex=False)
        .str.replace(",", ".", regex=False)
    )
    df_long["Valor"] = pd.to_numeric(df_long["Valor"], errors="coerce")

    print(f" Dataset 'cultura' cargado correctamente: {df_long.shape}")
    print(df_long.head(200))
    return df_long


# Ejemplo de uso
if __name__ == "__main__":
    df_cultura = cargar_cultura_limpia("raw_data/numero de intereses culturales.csv")

dfs["cultura"] = df_cultura

 Dataset 'cultura' cargado correctamente: (3580, 4)
    Código territorio             Territorio   Año  Valor
0                0014           Acebeda (La)  <NA>    NaN
1                0029                Ajalvir  <NA>    NaN
2                0035      Alameda del Valle  <NA>    NaN
3                0040            Ãlamo (El)  <NA>    NaN
4                0053     AlcalÃ¡ de Henares  <NA>    NaN
..                ...                    ...   ...    ...
195              0170                 Batres  2006    2.0
196              0186  Becerril de la Sierra  2006    0.0
197              0199       Belmonte de Tajo  2006    0.0
198              0210          Berrueco (El)  2006    1.0
199              0203     Berzosa del Lozoya  2006    0.0

[200 rows x 4 columns]


## Limpieza y tranformación antes de unirse

In [4]:
def pipeline_general(df, year_col="Año", code_col="Código territorio", value_col="Valor", last_year=True, to_int=True, rename_value=None):
    """Función general para transformar datasets con estructura común"""
    df = df.copy()
    
    # Filtrar por último año
    if last_year and year_col in df.columns:
        # garantizar que el año se compare como numérico (puede haber NaNs)
        df_years = pd.to_numeric(df[year_col], errors="coerce")
        max_year = df_years.max()
        df = df[df_years == max_year]
    
    # Convertir código de municipio a entero nullable (soporta NA)
    if code_col in df.columns:
        df["cod_municipio"] = pd.to_numeric(df[code_col], errors="coerce").astype("Int64")
    
    # Renombrar columna de valor (si se pide)
    if rename_value and value_col in df.columns:
        df = df.rename(columns={value_col: rename_value})
    
    # determinar nombre final de la columna de valor
    target_value_col = rename_value if rename_value else value_col
    
    # Convertir columna valor a numérica (soporta coma como decimal si ya limpia; en caso contrario intenta conversión directa)
    if target_value_col in df.columns:
        # si es object, intentar limpiar separadores comunes (puntos miles, comas decimales)
        if df[target_value_col].dtype == object:
            # Reemplazos no destructivos: quitar espacios y convertir comas a puntos
            df[target_value_col] = (
                df[target_value_col]
                .astype(str)
                .str.replace(r"\s+", "", regex=True)
                .str.replace(".", "", regex=False)  # eliminar separadores de miles
                .str.replace(",", ".", regex=False)  # coma decimal -> punto
            )
        df[target_value_col] = pd.to_numeric(df[target_value_col], errors="coerce")
    
    return df

# --- Función específica para vivienda (más robusta) ---
def pipeline_vivienda(df):
    df = df.copy()

    # Si ya está procesado (columnas esperadas), devolver versión normalizada
    if {"cod_postal", "municipio", "IPVA"}.issubset(df.columns):
        df["cod_postal"] = pd.to_numeric(df["cod_postal"], errors="coerce").astype("Int64")
        df["IPVA"] = pd.to_numeric(df["IPVA"], errors="coerce")
        df["municipio"] = df["municipio"].astype(str).str.strip()
        return df[["cod_postal", "municipio", "IPVA"]].reset_index(drop=True)

    # Localizar columna que contiene municipio (acepta varias variantes)
    muni_col = None
    for c in ["Municipio", "Territorio", "municipio", "territorio"]:
        if c in df.columns:
            muni_col = c
            break
    if muni_col is None:
        raise KeyError("No se encontró columna de municipio. Buscando 'Municipio' o 'Territorio'.")

    # Localizar columna de periodo/año si existe
    period_col = None
    for c in ["Periodo", "Año", "Year"]:
        if c in df.columns:
            period_col = c
            break
    # Filtrar por último año ignorando valores nulos en periodo
    if period_col is not None:
        periodo_num = pd.to_numeric(df[period_col], errors="coerce")
        valid_periods = periodo_num.dropna()
        if not valid_periods.empty:
            last_year = valid_periods.max()
            df = df[periodo_num == last_year]

    # Normalizar texto de la columna municipio y separar código y nombre
    s = df[muni_col].astype(str).fillna("").str.strip()
    # reemplazar guiones largos por espacio y limpiar espacios extra
    s_clean = s.str.replace(r"[-–—]+", " ", regex=True).str.strip()
    parts = s_clean.str.split(r"\s+", n=1, expand=True)

    # Extraer dígitos iniciales como código postal/código municipio cuando existan
    first_part_digits = parts[0].str.extract(r"(\d+)", expand=False)
    df["cod_postal"] = pd.to_numeric(first_part_digits, errors="coerce").astype("Int64")

    # Si no hay segunda parte, usar la primera (útil si ya solo contiene nombre)
    df["municipio"] = parts[1].where(parts[1].notna(), parts[0]).astype(str).str.strip()

    # Localizar columna de valor (Total/IPVA u otras variantes)
    value_col = None
    for c in ["Total", "IPVA", "Valor", "Importe"]:
        if c in df.columns:
            value_col = c
            break
    if value_col is None:
        # intentar usar la primera columna numérica razonable
        numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()
        # excluir columnas que claramente no son el valor buscado
        for excl in ["cod_postal"]:
            if excl in numeric_cols:
                numeric_cols.remove(excl)
        if numeric_cols:
            value_col = numeric_cols[0]

    if value_col is None:
        raise KeyError("No se encontró columna de valor (Total/IPVA/Valor) en el dataset de vivienda.")

    # Limpiar formato del valor y convertir a numérico
    df["IPVA"] = (
        df[value_col]
        .astype(str)
        .str.replace(r"\s+", "", regex=True)
        .str.replace(".", "", regex=False)
        .str.replace(",", ".", regex=False)
    )
    df["IPVA"] = pd.to_numeric(df["IPVA"], errors="coerce")

    return df[["cod_postal", "municipio", "IPVA"]].reset_index(drop=True)

# --- Función específica para cultura (maneja años nulos) ---
def pipeline_cultura(df):
    df = df.copy()
    # Filtrar por último año ignorando valores nulos en 'Año'
    if "Año" in df.columns:
        year_num = pd.to_numeric(df["Año"], errors="coerce")
        valid_years = year_num.dropna()
        if not valid_years.empty:
            last_year = valid_years.max()
            df = df[year_num == last_year]
        # si no hay años válidos, no filtramos (mantenemos todo)
    # convertir código a entero nullable (si existe)
    if "Código territorio" in df.columns:
        df["cod_municipio"] = pd.to_numeric(df["Código territorio"], errors="coerce").astype("Int64")
    else:
        # intentar encontrar columna parecida si cambia el nombre
        for c in df.columns:
            if "codigo" in c.lower() or "territorio" in c.lower():
                df["cod_municipio"] = pd.to_numeric(df[c], errors="coerce").astype("Int64")
                break
    # asegurar Valor numérico si existe
    if "Valor" in df.columns:
        df["Valor"] = pd.to_numeric(df["Valor"], errors="coerce")
    return df[["cod_municipio", "Valor"]]

# Aplicar pipelines solo si existen las claves en dfs (evita KeyError)
if "centros_salud" in dfs:
    dfs["centros_salud"] = pipeline_general(dfs["centros_salud"], year_col="Año", code_col="Código territorio", value_col="Valor", rename_value="centros_salud")
if "farmacias" in dfs:
    dfs["farmacias"] = pipeline_general(dfs["farmacias"], rename_value="farmacias")
if "paro_total" in dfs:
    dfs["paro_total"] = pipeline_general(dfs["paro_total"], rename_value="paro_total")
if "paro_por100" in dfs:
    dfs["paro_por100"] = pipeline_general(dfs["paro_por100"], rename_value="paro_100")
if "distancia" in dfs:
    dfs["distancia"] = pipeline_general(dfs["distancia"], rename_value="distancia_capital")
if "vivienda" in dfs:
    dfs["vivienda"] = pipeline_vivienda(dfs["vivienda"])
if "afiliados" in dfs:
    dfs["afiliados"] = pipeline_general(dfs["afiliados"], rename_value="afiliados")
if "alumnos" in dfs:
    dfs["alumnos"] = pipeline_general(dfs["alumnos"], rename_value="alumnos")
if "cultura" in dfs:
    dfs["cultura"] = pipeline_cultura(dfs["cultura"])

In [5]:
#  Revisar que todo se haya transformado correctamente
for name, df in dfs.items():
    print(f"\n{name.upper()} — Filas: {len(df)} | Columnas: {df.shape[1]}")
    print(df.head(5))


MUNICIPIOS_BASE — Filas: 155 | Columnas: 6
   cod_municipio          municipio   latitud  longitud    altitud  poblacion
0             14       Acebeda (La)  41.08697 -3.624634  1266.5420       68.0
1             29            Ajalvir  40.53437 -3.481002   680.1722     4946.0
2             35  Alameda del Valle  40.91790 -3.843788  1109.9340      256.0
3             40         Álamo (El)  40.22972 -3.992688   606.2238    10413.0
4             88   Aldea del Fresno  40.32399 -4.202217   476.7994     3422.0

CENTROS_SALUD — Filas: 573 | Columnas: 8
    Año      Tipo territorio  Código territorio           Territorio  \
0  2024  Comunidad de Madrid                NaN  Comunidad de Madrid   
1  2024           Municipios               14.0         Acebeda (La)   
2  2024           Municipios               29.0              Ajalvir   
3  2024           Municipios               35.0    Alameda del Valle   
4  2024           Municipios               40.0           Álamo (El)   

             

## Unión de datasets: DATASET COMPLETO

In [6]:
# === FUNCIONES AUXILIARES ===

def preparar_afiliados(df):
    """Convierte afiliados (one-to-many) a formato ancho por rama de actividad.
    Normaliza la columna 'Rama de actividad' (minúsculas, sin espacios ni caracteres inválidos)
    antes del pivot.
    """
    df = df.copy()
    # Normalizar texto
    df["Rama de actividad"] = df["Rama de actividad"].astype(str).str.strip().str.lower()
    # Reemplazar espacios por guiones bajos y eliminar caracteres no alfanuméricos/guión bajo
    df["rama_norm"] = (
        df["Rama de actividad"]
        .str.replace(r"\s+", "_", regex=True)
        .str.replace(r"[^\w]", "", regex=True)
    )
    # Asegurar que 'afiliados' es numérico
    df["afiliados"] = pd.to_numeric(df.get("afiliados"), errors="coerce").fillna(0)
    # Pivotear
    df_pivot = df.pivot_table(
        index="cod_municipio",
        columns="rama_norm",
        values="afiliados",
        aggfunc="sum",
        fill_value=0,
    ).reset_index()
    # Renombrar columnas de salida
    df_pivot.columns = ["cod_municipio"] + [f"afiliados_{c}" for c in df_pivot.columns[1:]]
    return df_pivot

def preparar_alumnos(df):
    """Agrupa por municipio para obtener el total de alumnos no universitarios."""
    df = df.copy()
    df["alumnos"] = pd.to_numeric(df["alumnos"], errors="coerce")
    df_agg = df.groupby("cod_municipio", as_index=False)["alumnos"].sum()
    df_agg = df_agg.rename(columns={"alumnos": "alumnos_total"})
    return df_agg

def preparar_cultura(df):
    df = df.copy()
    df = df.rename(columns={"Valor": "cultura"})
    return df[["cod_municipio", "cultura"]]

def preparar_centros_salud(df):
    df = df.copy()
    # Filtrar solo las filas donde la columna 'Tipo' indique 'Centros de salud'
    if "Tipo" in df.columns:
        df = df[df["Tipo"].astype(str).str.strip().str.lower() == "centros de salud"].copy()
    df = df.rename(columns={"centros_salud": "centros_salud_10mil"})
    return df[["cod_municipio", "centros_salud_10mil"]]

def preparar_farmacias(df):
    df = df.copy()
    df = df.rename(columns={"farmacias": "farmacias_10mil"})
    return df[["cod_municipio", "farmacias_10mil"]]

def preparar_vivienda(df):
    df = df.copy()
    df = df.rename(columns={"municipio": "municipio", "IPVA": "IPVA"})
    df["municipio"] = df["municipio"].str.strip().str.lower()
    return df[["municipio", "IPVA"]]

def preparar_municipios_base(df):
    df = df.copy()
    return df[["cod_municipio", "municipio", "latitud", "longitud", "altitud", "poblacion"]]

# === PIPELINE DE UNIÓN ===
def unir_todos(dfs, output_path="df_final.csv"):
    # Base: municipios
    df_final = preparar_municipios_base(dfs["municipios_base"])

    def merge(df_dest, name, how="left", on="cod_municipio"):
        if name not in dfs:
            print(f"⚠️ {name} no encontrado, se omite.")
            return df_dest

        print(f" Uniendo {name}...")
        df_src = dfs[name]
        result = df_dest.merge(df_src, on=on, how=how, suffixes=("", f"_{name}"))

        # Si se crean columnas municipio_x / municipio_y, mantener solo la de municipios_base
        for col in ["municipio_y", f"municipio_{name}"]:
            if col in result.columns:
                result.drop(columns=[col], inplace=True)
        if "municipio_x" in result.columns:
            result.rename(columns={"municipio_x": "municipio"}, inplace=True)

        return result

    # Preparar datasets específicos
    if "centros_salud" in dfs:
        dfs["centros_salud"] = preparar_centros_salud(dfs["centros_salud"])
    if "farmacias" in dfs:
        dfs["farmacias"] = preparar_farmacias(dfs["farmacias"])
    if "cultura" in dfs:
        dfs["cultura"] = preparar_cultura(dfs["cultura"])
    if "afiliados" in dfs:
        dfs["afiliados"] = preparar_afiliados(dfs["afiliados"])
    if "alumnos" in dfs:
        dfs["alumnos"] = preparar_alumnos(dfs["alumnos"])
    if "vivienda" in dfs:
        dfs["vivienda"] = preparar_vivienda(dfs["vivienda"])

    # Merge progresivo por cod_municipio
    for name in [
        "centros_salud",
        "farmacias",
        "google_places",
        "paro_total",
        "paro_por100",
        "distancia",
        "afiliados",
        "alumnos",
        "cultura",
    ]:
        df_final = merge(df_final, name)

    # Unir vivienda por municipio (texto)
    if "vivienda" in dfs:
        df_viv = dfs["vivienda"]
        df_final["municipio_norm"] = df_final["municipio"].str.strip().str.lower()
        df_viv["municipio"] = df_viv["municipio"].astype(str).str.strip().str.lower()

        df_final = df_final.merge(
            df_viv,
            left_on="municipio_norm",
            right_on="municipio",
            how="left"
        )
        df_final.drop(columns=["municipio_norm", "municipio_y"], inplace=True, errors="ignore")
        df_final.rename(columns={"municipio_x": "municipio"}, inplace=True)

    print(f"Filas: {len(df_final)} | Columnas: {df_final.shape[1]}")
    return df_final

# Unir y ver resultado final
df_final = unir_todos(dfs)
# Definir columnas finales a conservar
final_columns = [
    "cod_municipio", "municipio", "latitud", "longitud", "altitud", "poblacion",
    "centros_salud_10mil", "farmacias_10mil",
    "n_gym", "gym_total_reviews", "gym_weighted_avg_rating", "gym_rating_min", "gym_rating_max",
    "n_school", "school_total_reviews", "school_weighted_avg_rating", "school_rating_min", "school_rating_max",
    "n_transport", "transport_total_reviews", "transport_weighted_avg_rating", "transport_rating_min", "transport_rating_max",
    "n_restaurant", "restaurant_total_reviews", "restaurant_weighted_avg_rating", "restaurant_rating_min", "restaurant_rating_max",
    "n_pharmacy", "pharmacy_total_reviews", "pharmacy_weighted_avg_rating", "pharmacy_rating_min", "pharmacy_rating_max",
    "paro_total", "paro_100", "distancia_capital",
    "alumnos_total", "cultura", "IPVA"
]

# Filtrar solo las columnas que queremos conservar
cols_to_keep = [c for c in df_final.columns if c in final_columns or c.startswith("afiliados_")]
df_final = df_final[cols_to_keep]
print(f"\n=== DATAFRAME FINAL UNIDO ===\nFilas: {len(df_final)} | Columnas: {df_final.shape[1]}")
print(df_final.head(10))
print(df_final.columns.tolist())


 Uniendo centros_salud...
 Uniendo farmacias...
 Uniendo google_places...
 Uniendo paro_total...
 Uniendo paro_por100...
 Uniendo distancia...
 Uniendo afiliados...
 Uniendo alumnos...
 Uniendo cultura...
Filas: 155 | Columnas: 64

=== DATAFRAME FINAL UNIDO ===
Filas: 155 | Columnas: 46
   cod_municipio          municipio   latitud  longitud    altitud  poblacion  \
0             14       Acebeda (La)  41.08697 -3.624634  1266.5420       68.0   
1             29            Ajalvir  40.53437 -3.481002   680.1722     4946.0   
2             35  Alameda del Valle  40.91790 -3.843788  1109.9340      256.0   
3             40         Álamo (El)  40.22972 -3.992688   606.2238    10413.0   
4             88   Aldea del Fresno  40.32399 -4.202217   476.7994     3422.0   
5             91             Algete  40.59642 -3.497902   712.8972    21134.0   
6            105          Alpedrete  40.65875 -4.023713   916.7029    15655.0   
7            112             Ambite  40.32920 -3.181895   667.33

# EDA (df_final)

In [7]:
# === EDA BÁSICO ===

# Ver dimensiones
print(" Dimensiones:", df_final.shape)

# Ver primeras filas
print("\n Vista previa:")
display(df_final.head())

# Tipos de datos
print("\n Tipos de datos:")
print(df_final.dtypes)

# Valores faltantes
print("\n Valores nulos por columna:")
print(df_final.isnull().sum().sort_values(ascending=False))

# Porcentaje de valores nulos
print("\n Porcentaje de nulos:")
print((df_final.isnull().mean() * 100).round(2).sort_values(ascending=False))

# Valores únicos
print("\n Número de valores únicos por columna:")
print(df_final.nunique().sort_values(ascending=False))

# Estadísticas numéricas
print("\n Resumen estadístico:")
display(df_final.describe().T)


 Dimensiones: (155, 46)

 Vista previa:


,cod_municipio,municipio,latitud,longitud,altitud,poblacion,centros_salud_10mil,farmacias_10mil,n_gym,gym_total_reviews,...,afiliados_agricultura_y_ganadería,afiliados_construcción,afiliados_minería_industria_y_energía,afiliados_otros_servicios,afiliados_servicios_a_empresas_y_financieros,afiliados_servicios_de_distribución_y_hostelería,afiliados_total,alumnos_total,cultura,IPVA
0,14,Acebeda (La),41.08697,-3.624634,1266.5420,68.0,0,0,1,0,...,0,1,2,8,8,11,30,0,0.0,NaN
1,29,Ajalvir,40.53437,-3.481002,680.1722,4946.0,0,2,20,1563,...,15,110,253,578,521,691,2168,1336,0.0,NaN
2,35,Alameda del Valle,40.91790,-3.843788,1109.9340,256.0,0,39,1,0,...,4,3,4,31,20,27,89,0,0.0,NaN
3,40,Álamo (El),40.22972,-3.992688,606.2238,10413.0,0,2,19,2401,...,22,371,319,1164,773,1122,3771,4244,0.0,NaN
4,88,Aldea del Fresno,40.32399,-4.202217,476.7994,3422.0,0,3,4,147,...,27,149,70,340,213,283,1082,566,0.0,NaN



 Tipos de datos:
cod_municipio                                         int64
municipio                                            object
latitud                                             float64
longitud                                            float64
altitud                                             float64
poblacion                                           float64
centros_salud_10mil                                   int64
farmacias_10mil                                       int64
n_gym                                                 int64
gym_total_reviews                                     int64
gym_weighted_avg_rating                             float64
gym_rating_min                                      float64
gym_rating_max                                      float64
n_restaurant                                          int64
restaurant_total_reviews                              int64
restaurant_weighted_avg_rating                      float64
restaurant_rating_min 

,count,mean,std,min,25%,50%,75%,max
cod_municipio,155.0,1050.483871,1059.854519,14.000000,474.000000,954.000000,1447.500000,9020.000000
latitud,155.0,40.569981,0.291154,40.068120,40.332945,40.547490,40.817990,41.133120
longitud,155.0,-3.688450,0.328369,-4.490765,-3.946887,-3.617723,-3.461919,-3.106115
altitud,155.0,830.122131,214.092959,476.799400,659.797400,765.679100,1011.697000,1436.419000
poblacion,155.0,6340.058065,8128.545833,68.000000,832.000000,2952.000000,8421.000000,38969.000000
centros_salud_10mil,155.0,0.270968,0.839834,0.000000,0.000000,0.000000,0.000000,6.000000
farmacias_10mil,155.0,6.187097,8.210531,0.000000,2.000000,3.000000,6.000000,50.000000
n_gym,155.0,10.483871,7.915425,0.000000,2.000000,10.000000,20.000000,20.000000
gym_total_reviews,155.0,1333.335484,5553.688383,0.000000,1.000000,312.000000,1200.000000,68031.000000
gym_weighted_avg_rating,119.0,4.429229,0.553318,1.000000,4.298924,4.476890,4.714090,5.000000


Algunos NaN vienen de municipios sin gimnasios, colegios, farmacias o transporte registrados (no porque falte el dato, sino porque el valor es inexistente). Otros son porque quizás tengan pocas instalaciones o nadie haya puesto ninguna reseña. Lo exploraremos más a fondo. 

En el caso de 'IPVA', que presenta un 83.23% de null values, plantearíamos quitar esa columna. Esto se debe a que la dataset original contiene datos de municipios de toda España, y muchos municipios, sobretodo pequeños, de Madrid no aparecían. Los municipios que no estaban presentes en el dataset originalmente los metían en una fila con valor municipio como 'Resto Madrid'con IPVA de 118,417. Es un valor cercano a la media de los municipios de los que sí tenemos datos, que es 118.078192.

In [8]:
# === GIMNASIOS ===
gym_cols = ["cod_municipio", "municipio", "n_gym", "gym_total_reviews", 
             "gym_weighted_avg_rating", "gym_rating_min", "gym_rating_max"]
print(" FILAS CON gym_rating nulo:\n")
display(df_final[df_final["gym_weighted_avg_rating"].isna()][gym_cols])

# === FARMACIAS ===
pharma_cols = ["cod_municipio", "municipio", "farmacias_10mil", 'n_pharmacy',
               "pharmacy_weighted_avg_rating", "pharmacy_rating_min", "pharmacy_rating_max"]
print("\n FILAS CON pharmacy_rating nulo:\n")
display(df_final[df_final["pharmacy_weighted_avg_rating"].isna()][pharma_cols])

# === ESCUELAS ===
school_cols = ["cod_municipio", "municipio", "n_school", "alumnos_total",
               "school_total_reviews", "school_weighted_avg_rating", 
               "school_rating_min", "school_rating_max"]
print("\n FILAS CON school_rating nulo:\n")
display(df_final[df_final["school_weighted_avg_rating"].isna()][school_cols])

# === TRANSPORTE ===
transport_cols = ["cod_municipio", "municipio", "n_transport",
                  "transport_total_reviews", "transport_weighted_avg_rating",
                  "transport_rating_min", "transport_rating_max"]
print("\n FILAS CON transport_rating nulo:\n")
display(df_final[df_final["transport_weighted_avg_rating"].isna()][transport_cols])


 FILAS CON gym_rating nulo:



,cod_municipio,municipio,n_gym,gym_total_reviews,gym_weighted_avg_rating,gym_rating_min,gym_rating_max
0,14,Acebeda (La),1,0,NaN,NaN,NaN
2,35,Alameda del Valle,1,0,NaN,NaN,NaN
7,112,Ambite,2,0,NaN,NaN,NaN
10,164,Atazar (El),0,0,NaN,NaN,NaN
15,203,Berzosa del Lozoya,0,0,NaN,NaN,NaN
17,246,Braojos,1,0,NaN,NaN,NaN
20,278,Buitrago del Lozoya,1,0,NaN,NaN,NaN
27,344,Canencia,1,0,NaN,NaN,NaN
28,357,Carabaña,1,0,NaN,NaN,NaN
32,395,Cervera de Buitrago,2,0,NaN,NaN,NaN



 FILAS CON pharmacy_rating nulo:



,cod_municipio,municipio,farmacias_10mil,n_pharmacy,pharmacy_weighted_avg_rating,pharmacy_rating_min,pharmacy_rating_max
10,164,Atazar (El),0,0,NaN,NaN,NaN
95,1185,Puebla de la Sierra,0,0,NaN,NaN,NaN
103,1261,Robregordo,0,0,NaN,NaN,NaN
116,1436,Somosierra,0,0,NaN,NaN,NaN



 FILAS CON school_rating nulo:



,cod_municipio,municipio,n_school,alumnos_total,school_total_reviews,school_weighted_avg_rating,school_rating_min,school_rating_max
10,164,Atazar (El),0,0,0,NaN,NaN,NaN
14,210,Berrueco (El),8,12,0,NaN,NaN,NaN
15,203,Berzosa del Lozoya,0,0,0,NaN,NaN,NaN
18,259,Brea de Tajo,7,14,0,NaN,NaN,NaN
23,301,Cabrera (La),11,2358,0,NaN,NaN,NaN
24,318,Cadalso de los Vidrios,9,880,0,NaN,NaN,NaN
28,357,Carabaña,9,412,0,NaN,NaN,NaN
30,376,Cenicientos,8,454,0,NaN,NaN,NaN
32,395,Cervera de Buitrago,1,22,0,NaN,NaN,NaN
45,552,Estremera,6,312,0,NaN,NaN,NaN



 FILAS CON transport_rating nulo:



,cod_municipio,municipio,n_transport,transport_total_reviews,transport_weighted_avg_rating,transport_rating_min,transport_rating_max
95,1185,Puebla de la Sierra,1,0,NaN,NaN,NaN
129,1570,Valdelaguna,21,0,NaN,NaN,NaN
144,1739,Villamanrique de Tajo,6,0,NaN,NaN,NaN


# Preproceso

Los municipios sin oferta (sin gym, farmacia, colegio o transporte) tendrán 0 → reflejando falta de servicio.

Los municipios con servicios pero sin reseñas tendrán valores medios representativos. Refleja disponibilidad educativa sin penalizar excesivamente por falta de reseñas.

In [9]:
# Copia del df para imputación
df_clean = df_final.copy()

# Drop IPVA
df_clean = df_clean.drop(columns=["IPVA"])

# --- GYMS ---
gym_mean = df_clean["gym_weighted_avg_rating"].mean(skipna=True)
df_clean.loc[df_clean["n_gym"] == 0, ["gym_weighted_avg_rating", "gym_rating_min", "gym_rating_max"]] = 0
df_clean["gym_weighted_avg_rating"] = df_clean["gym_weighted_avg_rating"].fillna(gym_mean)
df_clean["gym_rating_min"] = df_clean["gym_rating_min"].fillna(gym_mean)
df_clean["gym_rating_max"] = df_clean["gym_rating_max"].fillna(gym_mean)

# --- FARMACIAS ---
df_clean.loc[df_clean["farmacias_10mil"] == 0, ["pharmacy_weighted_avg_rating", "pharmacy_rating_min", "pharmacy_rating_max"]] = 0

# --- SCHOOL ---
school_mean = df_clean.loc[df_clean["n_school"] > 0, "school_weighted_avg_rating"].mean()
df_clean["school_weighted_avg_rating"] = df_clean["school_weighted_avg_rating"].fillna(school_mean)
df_clean["school_rating_min"] = df_clean["school_rating_min"].fillna(school_mean)
df_clean["school_rating_max"] = df_clean["school_rating_max"].fillna(school_mean)

# --- TRANSPORT ---
transport_mean = df_clean.loc[df_clean["n_transport"] > 0, "transport_weighted_avg_rating"].mean()
df_clean["transport_weighted_avg_rating"] = df_clean["transport_weighted_avg_rating"].fillna(transport_mean)
df_clean["transport_rating_min"] = df_clean["transport_rating_min"].fillna(transport_mean)
df_clean["transport_rating_max"] = df_clean["transport_rating_max"].fillna(transport_mean)

# Verificar que no quedan nulos
print(df_clean.isna().sum().sort_values(ascending=False))

cod_municipio                                       0
municipio                                           0
latitud                                             0
longitud                                            0
altitud                                             0
poblacion                                           0
centros_salud_10mil                                 0
farmacias_10mil                                     0
n_gym                                               0
gym_total_reviews                                   0
gym_weighted_avg_rating                             0
gym_rating_min                                      0
gym_rating_max                                      0
n_restaurant                                        0
restaurant_total_reviews                            0
restaurant_weighted_avg_rating                      0
restaurant_rating_min                               0
restaurant_rating_max                               0
n_pharmacy                  

# Cálculo de nuevas columnas
Para evitar sesgos derivados del tamaño poblacional, quiero transformar las variables absolutas en indicadores relativos, expresados por cada X habitantes (por ejemplo, por cada 10.000 habitantes). Esto permite comparar municipios de distinto tamaño con métricas homogéneas.


In [10]:
# Lista de columnas que representan valores absolutos y deben normalizarse
cols_to_scale = [
    # Equipamientos y números absolutos
    "n_gym", "n_restaurant", "n_pharmacy", "n_school", 
    # Indicadores socioeconómicos
    "paro_total",  # Normalizamos y eliminamos paro_100 (inconsistente con el resto)
    "afiliados_construcción",
    "afiliados_agricultura_y_ganadería",
    "afiliados_minería_industria_y_energía",
    "afiliados_servicios_a_empresas_y_financieros",
    "afiliados_servicios_de_distribución_y_hostelería",
    "afiliados_otros_servicios",
    "afiliados_total",

    # Sistema educativo
    "alumnos_total"
]    

# Esto asegura comparabilidad entre municipios y elimina sesgo por tamaño poblacional
for col in cols_to_scale:
    df_clean[f"{col}_10mil"] = df_clean[col] / df_clean["poblacion"] * 10000

Una vez que los indicadores están por 10.000 habitantes, se eliminan las absolutas: n_school, n_restaurant, n_gym, etc.

In [11]:
cols_to_drop = [
    # Eliminamos recuentos absolutos porque ya tenemos la versión normalizada
    *cols_to_scale,

    # Eliminamos indicadores antiguos o inconsistentes
    "paro_100",              # Tenía un denominador distinto → inconsistente
    "farmacias_10mil"        # Se reemplaza por n_pharmacy normalizado (más actualizado)
]

df_clean = df_clean.drop(columns=cols_to_drop, errors="ignore")

In [12]:
# Guardar CSV
output_path = "dataset/df_clean.csv"
df_clean.to_csv(output_path, index=False, encoding="utf-8-sig")

# Cálculo interno 

**Diseño de Categorías de Bienestar Municipal**

Con el objetivo de evaluar municipios de menos de 50 000 habitantes y facilitar su redistribución poblacional manteniendo la calidad de vida, se han establecido seis dimensiones analíticas principales:

1. Sanidad: incluye indicadores sobre la densidad de centros de salud y farmacias, junto con la calidad percibida de estos servicios. Representa la accesibilidad y eficiencia de la atención médica básica.

2. Educación: mide la disponibilidad y valoración de los centros educativos no universitarios, así como el número total de alumnos matriculados. Esta dimensión refleja el nivel educativo local y su capacidad de retención de familias jóvenes.

3. Transporte y Conectividad: valora la existencia y calidad de los medios de transporte público, junto con la distancia a la capital provincial. Permite analizar la accesibilidad del municipio y su integración territorial.

4. Economía y Empleo: agrupa variables sobre el mercado laboral, la tasa de paro y la diversificación productiva. Evalúa la fortaleza económica y las oportunidades de empleo en el municipio.

5. Bienestar: se centra en la disponibilidad y calidad de infraestructuras orientadas al cuidado personal y la salud física, como gimnasios y centros deportivos. Representa el grado de fomento de hábitos saludables y la vida activa.

6. Ocio y Cultura: analiza la oferta gastronómica y cultural, incluyendo restaurantes y actividades culturales, como indicadores de vitalidad social, recreación y atractivo local.

Estas seis categorías permitirán calcular posteriormente un índice sintético de bienestar municipal. Cada bloque podrá ponderarse según las preferencias de los usuarios, generando un modelo de recomendación de municipios que equilibre servicios, oportunidades y calidad de vida.

## Por ejemplo: Índice Sanidad

Evaluar la disponibilidad y calidad del sistema sanitario local, considerando tanto el acceso físico a servicios como la valoración de los usuarios.

- Disponibilidad: centros_salud_10mil -> Centros de salud por cada 10.000 habitantes (↑ Más es mejor)
- Disponibilidad: farmacias_10mil -> Farmacias por cada 10.000 habitantes (↑ Más es mejor)
- Cantidad:	n_pharmacy -> Número total de farmacias registradas	(↑ Más es mejor)
- Calidad percibida:	pharmacy_weighted_avg_rating->	Media ponderada de valoraciones en Google Maps	(↑ Más es mejor)
- Calidad percibida:	pharmacy_rating_min -> Valoración mínima observada	(↑ Más es mejor)
- Calidad percibida:	pharmacy_rating_max ->	Valoración máxima observada	(↑ Más es mejor)

Y así pero con el resto de categorías restantes, que son la Educación, Transporte y Conectividad, Economía y Empleo, Bienestar, Ocio y Cultura.

## Cálculo de los Índices

### Definición de variables positivas y negativas

- Positivas: cuanto mayores, mejor (más farmacias, más escuelas, mejor rating, etc.).

- Negativas: cuanto menores, mejor (paro y distancia a la capital).

In [13]:
# Definir variables positivas y negativas para escalado

# VARIABLES POSITIVAS
positive_vars = [
    # Salud
    "centros_salud_10mil",
    "n_pharmacy_10mil", "pharmacy_weighted_avg_rating",

    # Gimnasios
    "n_gym_10mil", "gym_weighted_avg_rating",

    # Restaurantes
    "n_restaurant_10mil", "restaurant_weighted_avg_rating",

    # Educación
    "n_school_10mil", "school_weighted_avg_rating",
    "alumnos_total_10mil",

    # Transporte
    "n_transport", "transport_weighted_avg_rating",

    # Cultura
    "cultura",

    # Mercado laboral (afiliados normalizados)
    "afiliados_agricultura_y_ganadería_10mil",
    "afiliados_construcción_10mil",
    "afiliados_minería_industria_y_energía_10mil",
    "afiliados_otros_servicios_10mil",
    "afiliados_servicios_a_empresas_y_financieros_10mil",
    "afiliados_servicios_de_distribución_y_hostelería_10mil",
    "afiliados_total_10mil",
]

# VARIABLES NEGATIVAS
negative_vars = [
    "paro_total_10mil",   # normalizada
    "distancia_capital"   # esta sí es absoluta y tiene sentido mantenerla
]

# Scaler 
scaler = MinMaxScaler()

### Cálculo de 'commute_score', distancia a capital

Variable de distancia a capital:

- mean = 55.963845	
- std	= 19.867583
- min	= 18.348
- 25%	= 40.9795
- 50%	= 51.894
- 75% = 68.928
- max = 103.62

Se sacan los umbrares para la función con los estadísticos.

In [14]:
# Usar percentiles (quartiles) de 'distancia_capital' como umbrales para el commute score
q = df_clean["distancia_capital"].dropna()
p25, p50, p75 = q.quantile([0.25, 0.50, 0.75]).tolist()
print(f"Percentiles distancia_capital -> 25%: {p25:.3f}, 50%: {p50:.3f}, 75%: {p75:.3f}")

def commute_score(x):
    """Score de commute basado en quartiles:
       <= 25% -> 1.0 (mejor)
       25-50% -> 0.66
       50-75% -> 0.33
       > 75% -> 0.0 (peor)
       NaN -> pd.NA
    """
    if pd.isna(x):
        return pd.NA
    if x <= p25:
        return 1.0
    elif x <= p50:
        return 0.66
    elif x <= p75:
        return 0.33
    else:
        return 0.0

# Aplicar commute_score sobre el DataFrame limpio que contiene 'distancia_capital'
df_clean['commute_score'] = df_clean['distancia_capital'].apply(commute_score)

# Añadir la variable a positivas
positive_vars += ["commute_score"]

# Hacer que 'df' apunte al dataframe de trabajo (df_clean) para las celdas siguientes
df = df_clean

Percentiles distancia_capital -> 25%: 40.980, 50%: 51.894, 75%: 68.928


### Escalado: facilita el cálculo de índices compuestos.

- Se usa MinMaxScaler() → lleva todo a valores entre 0 y 1, permitiendo comparar diferentes magnitudes.


In [15]:
df_scaled = df.copy()

# scale all variables to [0, 1]
df_scaled[positive_vars + negative_vars] = scaler.fit_transform(
    df[positive_vars + negative_vars]
)

# invert negative variables
df_scaled[negative_vars] = 1 - df_scaled[negative_vars]
df_scaled.head(3)

,cod_municipio,municipio,latitud,longitud,altitud,poblacion,centros_salud_10mil,gym_total_reviews,gym_weighted_avg_rating,gym_rating_min,...,paro_total_10mil,afiliados_construcción_10mil,afiliados_agricultura_y_ganadería_10mil,afiliados_minería_industria_y_energía_10mil,afiliados_servicios_a_empresas_y_financieros_10mil,afiliados_servicios_de_distribución_y_hostelería_10mil,afiliados_otros_servicios_10mil,afiliados_total_10mil,alumnos_total_10mil,commute_score
0,14,Acebeda (La),41.08697,-3.624634,1266.5420,68.0,0.0,0,0.885846,4.429229,...,0.611878,0.198292,0.000000,0.286096,0.664554,0.919371,0.494118,0.786851,0.000000,0.0
1,29,Ajalvir,40.53437,-3.481002,680.1722,4946.0,0.0,1563,0.887614,3.000000,...,0.715195,0.299884,0.042459,0.497574,0.588027,0.770407,0.489820,0.780155,0.076831,1.0
2,35,Alameda del Valle,40.91790,-3.843788,1109.9340,256.0,0.0,0,0.885846,4.429229,...,0.335473,0.158014,0.218750,0.151989,0.418848,0.539152,0.512988,0.566555,0.000000,0.0


### Agrupación por categorías

- Cada categoría representa una dimensión del bienestar local.

- Cada una combina variables relacionadas.


In [16]:
categories = {
    "sanidad": [
        "centros_salud_10mil",
        "n_pharmacy_10mil",
        "pharmacy_weighted_avg_rating",
    ],

    "educacion": [
        "n_school_10mil",
        "school_weighted_avg_rating",
        "alumnos_total_10mil",
    ],

    "transporte_conectividad": [
        "n_transport",
        "transport_weighted_avg_rating",
        "commute_score",          # ya es puntuación 0–1, no hace falta escalar
    ],

    "economia_empleo": [
        "paro_total_10mil",
        "afiliados_agricultura_y_ganadería_10mil",
        "afiliados_construcción_10mil",
        "afiliados_minería_industria_y_energía_10mil",
        "afiliados_servicios_de_distribución_y_hostelería_10mil",
        "afiliados_servicios_a_empresas_y_financieros_10mil",
        "afiliados_otros_servicios_10mil",
        "afiliados_total_10mil",
    ],

    "bienestar": [
        "n_gym_10mil",
        "gym_weighted_avg_rating",
    ],

    "ocio_cultura": [
        "cultura",
        "n_restaurant_10mil",
        "restaurant_weighted_avg_rating",
    ],
}


### Pesos dentro de cada categpría
Asigna importancia relativa a cada variable dentro de su categoría.

- CÁLCULO AUTOMÁTICO DE PESOS CON PCA


Con el objetivo de determinar la importancia relativa de cada variable dentro de su respectiva categoría (por ejemplo, dentro de Sanidad o Educación), se ha aplicado un Análisis de Componentes Principales (PCA). Esta técnica permite asignar pesos de manera objetiva y basada en los datos, evitando sesgos derivados de decisiones arbitrarias.

En este análisis, el PCA se aplicó por categoría temática (Sanidad, Educación, Transporte y Conectividad, Economía y Empleo, Bienestar, Ocio y Cultura).
Cada grupo de variables fue previamente escalado a un rango común [0, 1] mediante normalización Min–Max, para garantizar que todas las variables tuvieran el mismo peso inicial independientemente de su escala original.

In [17]:
category_weights_pca = {}

for cat, vars_cat in categories.items():
    subset = df_scaled[vars_cat].dropna()
    if subset.shape[1] > 1:
        pca = PCA(n_components=1)
        pca.fit(subset)
        weights = np.abs(pca.components_[0])
        weights = weights / weights.sum()  # normalizar para que sumen 1
        category_weights_pca[cat] = dict(zip(vars_cat, weights))
    else:
        # si solo hay una variable, el peso es 1
        category_weights_pca[cat] = {vars_cat[0]: 1.0}

print("Pesos estimados con PCA por categoría:\n")
for cat, w in category_weights_pca.items():
    print(f"{cat}:")
    for var, weight in w.items():
        print(f"   {var:<45} → {weight:.3f}")
    print()

Pesos estimados con PCA por categoría:

sanidad:
   centros_salud_10mil                           → 0.046
   n_pharmacy_10mil                              → 0.228
   pharmacy_weighted_avg_rating                  → 0.726

educacion:
   n_school_10mil                                → 0.311
   school_weighted_avg_rating                    → 0.626
   alumnos_total_10mil                           → 0.063

transporte_conectividad:
   n_transport                                   → 0.040
   transport_weighted_avg_rating                 → 0.231
   commute_score                                 → 0.729

economia_empleo:
   paro_total_10mil                              → 0.111
   afiliados_agricultura_y_ganadería_10mil       → 0.114
   afiliados_construcción_10mil                  → 0.004
   afiliados_minería_industria_y_energía_10mil   → 0.073
   afiliados_servicios_de_distribución_y_hostelería_10mil → 0.204
   afiliados_servicios_a_empresas_y_financieros_10mil → 0.236
   afiliados_otros_servici

- Índices con pesos definidos manualmente (Otra forma)

Los pesos son basados en encuestas a usuarios. Se realizó una serie de entrevistas a personas de distintas edades y oficios sobre las categorías. 

In [18]:
# category_weights = {
#     "sanidad": {
#         "centros_salud_10mil": 0.35,
#         "n_pharmacy_10mil": 0.45,
#         "pharmacy_weighted_avg_rating": 0.20
#     },
#     "educacion": {
#         "school_weighted_avg_rating": 0.5,
#         "n_school_10mil": 0.3,
#         "alumnos_total_10mil": 0.2
#     },
#     "transporte_conectividad": {
#         "n_transport_10mil": 0.4,
#         "transport_weighted_avg_rating": 0.3,
#         "commute_score": 0.3
#     },
#     "economia_empleo": {
#         "paro_total_10mil": 0.6,
#         "afiliados_total_10mil": 0.4
#     },
#     "bienestar": {
#         "n_gym_10mil": 0.4,
#         "gym_weighted_avg_rating": 0.6
#     },
#     "ocio_cultura": {
#         "cultura_10mil": 0.3,
#         "n_restaurant_10mil": 0.3,
#         "restaurant_weighted_avg_rating": 0.4
#     }
# }


### Cálculo de los Índices y exportación de resultados

In [19]:
# CÁLCULO DE LOS ÍNDICES POR CATEGORÍA 

category_indices = pd.DataFrame(index=df_scaled.index)
for category, vars_cat in categories.items():
    weights = category_weights_pca[category]
    weighted_sum = sum(df_scaled[v] * w for v, w in weights.items())
    category_indices[category] = weighted_sum

# CÁLCULO DEL ÍNDICE GLOBAL 

df_final_joined = pd.concat(
    [df_scaled[['cod_municipio', 'municipio', 'latitud', 'longitud', 'altitud']], category_indices],
    axis=1
)

df_final_joined["overall_index"] = category_indices.mean(axis=1)

### Descripciones de pueblos

In [20]:
def rename(text):
    if "(El)" in text:
        text = text.replace("(El)", "").strip()
        text = "El " + text
    elif "(La)" in text:
        text = text.replace("(La)", "").strip()
        text = "La " + text
    return text

df_final_joined["municipio"] = df_final_joined["municipio"].apply(rename)

In [21]:
# Configura el idioma y un User-Agent (es buena práctica)
wiki_es = wikipediaapi.Wikipedia(
    language='es',
    user_agent='MiProyectoDeEjemplo (miemail@ejemplo.com)'
)

# columna Descripcion que contiene la descripcion del pueblo que se obtiene de la llamada a la API de Wikipedia cogiendo el primer parrafo.
def obtener_descripcion_wikipedia(municipio):
    """Obtiene la descripción del municipio desde Wikipedia."""
    pagina = wiki_es.page(municipio + " (Madrid)")
    if pagina.exists():
        text = pagina.summary
        text += pagina.sections[0].text if pagina.sections else ""
        return text
    else:
        return "Descripción no disponible."

In [22]:
# por municipio en df_final_joined, obtener la descripcion y añadirla a una nueva columna 'Descripcion'
df_final_joined['Descripcion'] = df_final_joined['municipio'].apply(obtener_descripcion_wikipedia)

In [23]:
# EXPORTAR RESULTADOS

df_final_joined.to_csv("website_data/df_indices_categorias.csv", index=False, encoding="utf-8-sig")
# print("Archivo 'df_indices_categorias.csv' generado con éxito.")